# Gerardo de Miguel González

**::GMG::** The aim of this notebook is to find a basic workflow for doing machine learning classification of an imbalanced dataset using best practices taken from other people's examples.

# Binary Classification Proof of Concept

## References

**::GMG::** These ones:

  - [STHDA](http://www.sthda.com/english/articles/36-classification-methods-essentials/149-penalized-logistic-regression-essentials-in-r-ridge-lasso-and-elastic-net/) Penalized Logistic Regression Essentials in R: Ridge, Lasso and Elastic Net by kassambara -- 11/03/2018 from  [Articles](http://www.sthda.com/english/articles/36-classification-methods-essentials/) - Classification Methods Essentials
  - [Stanford](https://web.stanford.edu/~hastie/glmnet/glmnet_alpha.html#log) Glmnet Vignette: Logistic Regression by [Trevor Hastie](https://web.stanford.edu/~hastie/) and Junyang Qian, Stanford June 26, 2014
  - [Lab Notebook](http://homepages.uc.edu/~lis6/Teaching/DM18Spring/Lab/lab5_logit.html) in [Data Mining I (BANA7046) 2018 Spring](http://homepages.uc.edu/~lis6/DM-18Spring.html) by [Shaobo Li](http://homepages.uc.edu/~lis6/), see [Variable Selection section](http://homepages.uc.edu/~lis6/Teaching/DM18Spring/Lab/lab5_logit.html#variable-selection).
  
**::NOTE::** The LASSO is not very good at handling variables which show correlation between them and thus can sometimes show very wild behaviors. (From [rstatisticsblog](https://www.rstatisticsblog.com/data-science-in-action/lasso-regression/)

## Libraries

In [ ]:
#::GMG::Helper function to install missing libraries before loading them
# https://stackoverflow.com/questions/4090169/elegant-way-to-check-for-missing-packages-and-install-them
using<-function(...) {
    libs<-unlist(list(...))
    req<-unlist(lapply(libs,require,character.only=TRUE))
    need<-libs[req==FALSE]
    if(length(need)>0){ 
        install.packages(need)
        lapply(need,require,character.only=TRUE)
    }
}

In [ ]:
#::NOTE::Downsizing to simpler packages
#using('tidyverse','caret','glmnet', 'mlbench')
# ... repr to fix size of graphics within jupyter noteboook  ...
# https://blog.revolutionanalytics.com/2015/09/resizing-plots-in-the-r-kernel-for-jupyter-notebooks.html
# ... [NO] crossval: Generic Functions for Cross Validation ...
#     Contains generic functions for performing cross validation and for computing diagnostic errors ...
# https://cran.r-project.org/web/packages/crossval/
# https://cran.r-project.org/web/packages/crossval/crossval.pdf
# ... plotmo: Plot a Model's Residuals, Response, and Partial Dependence Plots ...
#       Plot model surfaces for a wide variety of models using partial dependence plots and other techniques. 
#       Also plot model residuals and other information on the model.
# https://cran.r-project.org/web/packages/plotmo/index.html
# https://cran.r-project.org/web/packages/plotmo/plotmo.pdf
# ... MLmetrics: Machine Learning Evaluation Metrics
#     A collection of evaluation metrics, including loss, score and utility functions, that measure regression, 
#     classification and ranking performance.
using('repr', 'mlbench', 'caTools', 'glmnet', 'plotmo', 'MLmetrics')

**::GMG::** `crossval` package first install output here:

```R
Loading required package: mlbench
Loading required package: caTools
Loading required package: glmnet
Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-18

Loading required package: crossval
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘crossval’”Installing package into ‘/home/jovyan/R/x86_64-pc-linux-gnu-library/3.6’
(as ‘lib’ is unspecified)
Loading required package: crossval
  1. TRUE
```

**::GMG::** `tidyverse` output first install here:

```R
Loading required package: tidyverse
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘tidyverse’”Loading required package: caret
Loading required package: lattice
Loading required package: ggplot2
Loading required package: glmnet
Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-18

Installing package into ‘/home/jovyan/R/x86_64-pc-linux-gnu-library/3.6’
(as ‘lib’ is unspecified)
also installing the dependencies ‘tinytex’, ‘rmarkdown’, ‘selectr’, ‘broom’, ‘dbplyr’, ‘modelr’, ‘reprex’, ‘rvest’

Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ tibble  2.1.3     ✔ purrr   0.3.2
✔ tidyr   0.8.3     ✔ dplyr   0.8.3
✔ readr   1.3.1     ✔ stringr 1.4.0
✔ tibble  2.1.3     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::accumulate() masks foreach::accumulate()
✖ tidyr::expand()     masks Matrix::expand()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()
✖ purrr::lift()       masks caret::lift()
✖ purrr::when()       masks foreach::when()
                           
    1. TRUE
``` 

**::GMG::** First installation of plotmo:

```R
Loading required package: plotmo
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘plotmo’”Installing package into ‘/home/jovyan/R/x86_64-pc-linux-gnu-library/3.6’
(as ‘lib’ is unspecified)
also installing the dependencies ‘plotrix’, ‘TeachingDemos’

Loading required package: plotmo
Loading required package: Formula
Loading required package: plotrix
Loading required package: TeachingDemos

    1. TRUE

```

## Dataset

In [ ]:
#::GMG::There are two versions in mlbench
# https://www.rdocumentation.org/packages/utils/versions/3.6.1/topics/data
#::NOTE::2nd version comes with zero outliers as NAs
utils::data(PimaIndiansDiabetes2)

In [ ]:
#::GMG::See first rows
# https://www.rdocumentation.org/packages/utils/versions/3.6.1/topics/head
#::NOTE::Some NAs show up!
utils::head(PimaIndiansDiabetes2)

## Exploratory Data Analysis

**::GMG::** The goal of this stage of our study is to get to know the data, their flaws and perhaps some clues to improve them.

In [ ]:
#::GMG::PimaIndiansDiabetes2 (say, v2) has the outliers from 
#       glucose, pressure, triceps, insulin and mass as NAs
str(PimaIndiansDiabetes2)

In [ ]:
#::GMG::NA counts yield the outliers (!)
#::NOTE::Should I standardize some of the features ('glucose', 'pressure', 'triceps', 'insuline', 'mass')?
#::NOTE::Should I do feature engineering on some of the features 
#        (i.e. age, p.e breaking it into intervals with labels)?
summary(PimaIndiansDiabetes2)

In [ ]:
#::GMG::Check for missing values ... I know they are marked witn 'NA' in mlbench 2nd dataset
sapply(PimaIndiansDiabetes2, function(x) sum(is.na(x)))

In [ ]:
#::GMG::Now I test the imbalance of the dataset
# http://metadatascience.com/2013/07/03/plotting-the-frequency-distribution-using-r/
table(PimaIndiansDiabetes2$diabetes)

In [ ]:
prop.table(table(PimaIndiansDiabetes2$diabetes))

In [ ]:
#::GMG::Plot the imbalance
# https://www.statmethods.net/advgraphs/parameters.html
# http://www.programmingr.com/content/positioning-charts-fig-and-fin/
#::NOTE::I don't know how to make the fig smaller and the margins tight (!?)
# https://blog.revolutionanalytics.com/2015/09/resizing-plots-in-the-r-kernel-for-jupyter-notebooks.html
options(repr.plot.width=4, repr.plot.height=3)
barplot(100*prop.table(table(PimaIndiansDiabetes2$diabetes)), 
        col = c('blue', 'red'),
        ylim = c(0,70),
        main = 'Outcome Imbalance')

## Preparing the data

In [ ]:
#::GMG:: (STHDA) Load the data and remove NAs
#::NOTE::I've already attached that dataset earlier
# data("PimaIndiansDiabetes2", package = "mlbench")
# https://stat.ethz.ch/R-manual/R-devel/library/stats/html/na.fail.html
# https://www.statmethods.net/input/missingdata.html
pima_clean <- stats::na.omit(PimaIndiansDiabetes2)

In [ ]:
#::GMG::Check for missing values ... No NAs after cleaning with na.omit()
sapply(pima_clean, function(x) sum(is.na(x)))

In [ ]:
# Inspect the data
# https://dplyr.tidyverse.org/reference/sample.html
#dplyr::sample_n(pima_clean, 3)
#::NOTE::Downsize use of tidyverse to R core functions
# https://www.rdocumentation.org/packages/utils/versions/3.6.1/topics/head
utils::head(pima_clean)

In [ ]:
#::NOTE::Originally 768 Observations!!
str(pima_clean)

## Stratified Train/Test split

In [ ]:
#::NOTE::Make it reproducible
set.seed(123)

In [ ]:
# Split the data into training and test set
#::NOTE::STHDA uses caret to do the stratified split and dplyr %>% notation
#        and I think It's a bit of overdoing it ...
# https://topepo.github.io/caret/data-splitting.html
# https://github.com/tidyverse/dplyr
#training_samples <- pima_clean$diabetes %>% 
#  caret::createDataPartition(p = 0.8, list = FALSE)
#::NOTE::... downsizing to caTools (that does a pretty good job, I think)
msk <- caTools::sample.split(Y = pima_clean$diabetes, SplitRatio = 0.8)

In [ ]:
#::NOTE::Do the actual split with the indices just computed
#train_data  <- pima_clean[training_samples, ]
#test_data <- pima_clean[-training_samples, ]
#::NOTE::downsizing to caTools
train_data <- pima_clean[msk,]  # use output of sample.split to ...
test_data  <- pima_clean[!msk,] # create train and test subse

In [ ]:
#::NOTE::See the split datasets
str(train_data)

In [ ]:
#::GMG::Now I test the imbalance of the dataset
# http://metadatascience.com/2013/07/03/plotting-the-frequency-distribution-using-r/
table(train_data$diabetes)

In [ ]:
#::GMG::Just to be sure proportions are kept ...
prop.table(table(train_data$diabetes))

In [ ]:
str(test_data)

In [ ]:
#::GMG::Now I test the imbalance of the dataset
# http://metadatascience.com/2013/07/03/plotting-the-frequency-distribution-using-r/
table(test_data$diabetes)

In [ ]:
#::GMG::Just to be sure proportions are kept ...
prop.table(table(test_data$diabetes))

## Logistic regression model

### model

In [ ]:
#::GMG::First off, I use stats:glm() to fit a baseline Logistic Regression Model
# https://www.rdocumentation.org/packages/stats/versions/3.6.1/topics/glm
full_model <- stats::glm(
    formula = diabetes ~., 
    data = train_data, 
    family = "binomial"
)

In [ ]:
#::GMG::Let's see a summary of the model I've just got
# https://www.rdocumentation.org/packages/stats/versions/3.6.1/topics/summary.glm
# https://www.youtube.com/watch?v=xl5dZo_BSJk
# 
summary(full_model)

In [ ]:
#::GMG::Selecting the statistically significant variables in an R glm model
# https://stackoverflow.com/questions/16153497/selecting-the-statistically-significant-variables-in-an-r-glm-model
toselect.x <- summary(full_model)$coeff[-1,4] < 0.05 # credit to kith
# select sig. variables
relevant.x <- names(toselect.x)[toselect.x == TRUE] 
# formula with only sig variables
sig.formula <- as.formula(paste("diabetes ~",paste(relevant.x, collapse= "+")))
cat('Relevant: ', relevant.x, '\n')
print (sig.formula)

**::PENDING::** <span style="color:red">Discussion here</span>

**::GMG::** Help ...

  - [Jeff Harmrick](https://www.youtube.com/watch?v=xl5dZo_BSJk) Understanding the Summary Output for a Logistic Regression in R, Published on Nov 14, 2012.
  - [Cross-Validated](https://stats.stackexchange.com/questions/232548/r-how-are-the-significance-codes-determined-when-summarizing-a-logistic-regres) R - How are the significance codes determined when summarizing a logistic regression model?, Apr 13 2017 at 12:44
  - [Cross-Validated](https://stats.stackexchange.com/questions/72258/how-to-interpret-the-significance-code) How to interpret the significance code?, Oct 9 2013 at 2:26
  - [Cross_validated](https://stats.stackexchange.com/questions/60074/wald-test-for-logistic-regression) Wald test for logistic regression, May 26 2013 at 20:40 (Check out the [answer](https://stats.stackexchange.com/a/60083))
  - [R Tutorial](http://www.r-tutor.com/elementary-statistics/logistic-regression/significance-test-logistic-regression) Significance Test for Logistic Regression
  - [StatQuest](https://www.youtube.com/watch?v=yIYKR4sgzI8) Logistic Regression, by Josh Starmer Published on 5 Mar 2018
  - [StatQuest](https://www.youtube.com/watch?v=vN5cNN2-HWE) Logistic Regression Details Pt1: Coefficients, by Josh Starmer, Published on Jun 4, 2018
  

In [ ]:
simpler_model <- stats::glm(
    formula = diabetes ~ glucose + mass + pedigree + age, 
    data = train_data, 
    family = "binomial"
)

In [ ]:
summary(simpler_model)

**::PENDING::** <span style="color:red">Discussion here</span>

### predict

In [ ]:
#::GMG::Make predictions (from STHDA)
# https://www.rdocumentation.org/packages/stats/versions/3.6.1/topics/predict.glm
probabilities_full <- predict(
    object = full_model, 
    newdata = test_data, 
    type = "response"
)
str(probabilities_full)

In [ ]:
#::GMG::Make class predictions (from STHDA)
#predicted_class_full <- ifelse(probabilities_full > 0.5, "pos", "neg")
#::NOTE::I make the result a factor with levels that match the taget variable
predicted_class_full <- as.factor(ifelse(probabilities_full > 0.5, 2, 1))
levels(predicted_class_full) <- c('neg', 'pos')
str(predicted_class_full)

In [ ]:
head(predicted_class_full)

In [ ]:
#::GMG::For the P/R curve I need a numeric 0/1 not class labels, I'm afraid
str(predicted_class_num)

In [ ]:
#::GMG::Check the simpler model (I don't expect mmuch ;)
probabilities_simpler <- predict(
    object = simpler_model, 
    newdata = test_data, 
    type = "response"
)
str(probabilities_simpler)

In [ ]:
# predicted_class_num <- ifelse(probabilities_full > 0.5, 1, 0)
#::NOTE::I make the result a factor with levels that match the taget variable
#predicted_class_simpler <- ifelse(probabilities_simpler > 0.5, "pos", "neg")
predicted_class_simpler <- as.factor(ifelse(probabilities_simpler > 0.5, 2, 1))
levels(predicted_class_simpler) <- c('neg', 'pos')
str(predicted_class_simpler)

In [ ]:
head(predicted_class_simpler)

### evaluation: confusion matrix and metrics

In [ ]:
#::GMG::'Raw' Model accuracy
observed_classes <- test_data$diabetes
str(observed_classes)

In [ ]:
head(observed_classes)

In [ ]:
cat('Accuracy Full Model: ', 
    mean(predicted_class_full == observed_classes),
   '\nAccuracy Simpler Model: ', 
    mean(predicted_class_simpler == observed_classes))

In [ ]:
#::GMG::Confusion Matrix AND metrics
# https://www.rdocumentation.org/packages/base/versions/3.6.1/topics/table
# https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/
# https://stackoverflow.com/questions/33621592/sum-of-antidiagonal-of-a-matrix
# https://stackoverflow.com/questions/14577412/how-to-convert-variable-object-name-into-string
confusion_matrix <- function (true, pred, beta = 1) {
    t <- table(true, pred)
    p <- t[2,2]/sum(t[,2])  #::NOTE::I assume 'positive class' is class2 (whatever it is)
    r <- t[2,2]/sum(t[2,])  #::NOTE::I assume 'positive class' is class2 (whatever it is)
    s <- t[1,1]/sum(t[1,])
    list(cm = t,
         accuracy = sum(diag(t))/sum(t), 
         precision = p,     #::NOTE::Precision of 'positive class'    
         recall = r,        #::NOTE::it's also called sensitivity ('positive class again')
         f1 = (1 + beta^2) * p * r/((beta^2 * p) + r),  #::NOTA::beta = 1 -> equal importance (class1, class2)
         specificity = s,   #::NOTE::proportion of actual 'negative class' that are detected as such
         balanced_accuracy = (r + s) / 2
        )
}

In [ ]:
#::GMG::Confusion Matrix
cf_full <- confusion_matrix(true = test_data$diabetes, pred = predicted_class_full)
print(cf_full$cm)

In [ ]:
#::GMG::Confusion Matrix of the 'simpler' model ... no difference!
cf_simpler <- confusion_matrix(true = test_data$diabetes, pred = predicted_class_simpler)
print(cf_simpler$cm)

In [ ]:
#::GMG:Pretty print the model metrics as a table/data frame/...
# https://cran.r-project.org/web/packages/tibble/vignettes/tibble.html
# tibble::tibble(metric = names(cf[-1]), pima_logit = cf[-1])
# https://www.r-bloggers.com/converting-a-list-to-a-data-frame/
#as.data.frame.list(x = cf[-1], row.names = c('pima_logit'))
# https://stackoverflow.com/questions/32059798/list-of-named-lists-to-data-frame
# https://stackoverflow.com/questions/10432993/named-list-to-from-data-frame
# https://www.rdocumentation.org/packages/base/versions/3.6.1/topics/as.data.frame
# https://www.rdocumentation.org/packages/base/versions/3.6.1/topics/names
#::NOTE::sensitivity (pos class) is recall
results <- as.data.frame(t(sapply(list(cf_full[-1], cf_simpler[-1]), rbind)),
                         row.names = c('full_model', 'simpler_model')
)
colnames(results) <- names(cf_full[-1])
# https://stackoverflow.com/questions/23217520/limiting-the-number-of-decimals-in-a-dataframe-r
print(results, digits = 3)

### evaluation: P/R Curve

In [ ]:
#::GMG::Compute the curve
# https://cran.r-project.org/web/packages/PRROC/PRROC.pdf pp. 7
# Soft-labeled classification problems:
# Each data point belongs to both of the two classes with a certain probability, where for each datapoint, these 
# two probabilities add up to 1.  In this case, the classification scores for all data points need to be provided 
# only once as scores.class0 and only the positive/foreground weights for each data point need to be provided in
# weights.class0, while the converse probability for the negative class is automatically set to 
# weights.class1 = 1.0 - weights.class0.
pr_curve_full <- PRROC::pr.curve(
                scores.class0 = predicted_class_num, 
                weights.class0 = probabilities_full,
                min.compute = T, 
                max.compute = T,
                rand.compute = T,
                curve = T
)

In [ ]:
#::GMG::See all things within the pr_1 object
print(pr_curve_full)

In [ ]:
#::GMG::Plot the curve
# https://cran.r-project.org/web/packages/PRROC/PRROC.pdf pp 3-4
options(repr.plot.width = 8, repr.plot.height = 6)
plot(pr_curve_full, rand.plot = T)

**::PENDING::** <span style="color:red">Discussion here</span>

**::NOTE::** Is this right?

In [ ]:
ls()

## Model with Lasso (L1 regularization)

### features/target split

In [ ]:
#::NOTE::The R function model.matrix() helps to create the matrix of predictors and also automatically 
#        converts categorical predictors to appropriate dummy variables, which is required for the glmnet() 
#        function.
# https://www.rdocumentation.org/packages/stats/versions/3.6.1/topics/model.matrix
# Dumy code categorical predictor variables
X_train <- stats::model.matrix(diabetes~., train_data)[,-1]

In [ ]:
# Convert the outcome (class) to a numerical variable
y_train <- ifelse(train_data$diabetes == "pos", 1, 0)

In [ ]:
#::NOTE::Check the results (a named 2D matrix of features)
str(X_train)

In [ ]:
#::NOTE::Check the results (a numeric 1/0 vector)
str(y_train)

In [ ]:
#::GMG::Do the same with test, of course
X_test <- model.matrix(diabetes ~., test_data)[,-1]

In [ ]:
str(X_test)

In [ ]:
y_test <- ifelse(test_data$diabetes == "pos", 1, 0)

In [ ]:
str(y_test)

### computing best lambda with cv

In [ ]:
#::NOTE::Make it reproducible
set.seed(123) 

In [ ]:
#::NOTE::In penalized regression, you need to specify a constant lambda to adjust the amount of 
#        the coefficient shrinkage. The best lambda for your data, can be defined as the lambda 
#        that minimize the cross-validation prediction error rate. This can be determined automatically 
#        using the function cv.glmnet()
# https://www.rdocumentation.org/packages/glmnet/versions/2.0-18/topics/cv.glmnet
#::NOTE::compute lasso regression by specifying the option alpha = 1
# Find the optimal value of lambda that minimizes the cross-validation error:
cv_lasso <- glmnet::cv.glmnet(x = X_train, 
                              y = y_train, 
                              alpha = 1,
                              type.measure = 'class', #::NOTE::misclassification error
                              family = 'binomial')

In [ ]:
#::NOTE::See graphically the best lambda
options(repr.plot.width = 8, repr.plot.height = 6)
plot(cv_lasso)

The plot displays the cross-validation *misclassification error* according to the log of lambda. The *left dashed vertical line* indicates that the log of the optimal value of lambda `lambda.min` is approximately -3, which is the one that minimizes the prediction error. This lambda value will give the most accurate model. The exact value of lambda can be viewed as follow:

In [ ]:
cat('Optimum lambda: ', cv_lasso$lambda.min)

Generally, the purpose of regularization is to balance accuracy and simplicity. This means, a model with the smallest number of predictors that also gives a good accuracy. To this end, the function `cv.glmnet()` finds also the value of lambda that gives the simplest model but also lies within one standard error of the optimal value of lambda. This value is called `lambda.1se`.

In [ ]:
cat('Simplest model within one sd of lambda optimum: ', cv_lasso$lambda.1se)

Using lambda.min as the best lambda, gives the following regression coefficients:

In [ ]:
# https://www.rdocumentation.org/packages/stats/versions/3.6.1/topics/coef (?)
stats::coef(cv_lasso, cv_lasso$lambda.min)

In [ ]:
#::NOTE:the "native" glmnet packaage coef() (same result)
glmnet::coef.cv.glmnet(cv_lasso, cv_lasso$lambda.min)

Using `lambda.1se` as the best lambda, gives the following regression coefficients:

In [ ]:
coef(cv_lasso, cv_lasso$lambda.1se)

**::PENDING::** <span style="color:red">Discussion here</span>

In [ ]:
#::GMG::Plot the shrinkage of the coefficients
# https://stackoverflow.com/questions/36656752/plotting-cv-glmnet-in-r~
#::NOTE::How do I put the feature names on that plot?
# https://stats.stackexchange.com/questions/133873/lasso-plot-label-lines-with-names-using-glmnet
# https://www.rdocumentation.org/packages/glmnet/versions/2.0-18/topics/plot.glmnet
options(repr.plot.width = 8, repr.plot.height = 6)
plot(cv_lasso$glmnet.fit, "lambda", label = TRUE)

### better coeff shrinkage plot

In [ ]:
#::GMG::Adhoc function intermediate fix
#::NOTE::Adapted from
# https://stackoverflow.com/a/43327018
lbs_fun <- function(lra, ...) {
  fit <- lra$glmnet.fit

  L=which(fit$lambda==lra$lambda.min)

  ystart <- sort(fit$beta[abs(fit$beta[,L])>0,L])
  labs <- names(ystart)
  #::NOTE:: max gap between biggest and smallest coefs at smallest lambda
  #         i.e., 64th (more than that yields an error ... given 100th lambda
  #         in original!)
  r <- range(fit$beta[,64]) 
  yfin <- seq(r[1],r[2],length=length(ystart))

  xstart<- log(lra$lambda.min)
  xfin <- xstart+1

  text(xfin+0.3,yfin,labels=labs,...)
  segments(xstart,ystart,xfin,yfin)
  # https://www.r-bloggers.com/r-add-vertical-line-to-a-plot/
  # https://www.statmethods.net/advgraphs/parameters.html
  abline(v = log(lra$lambda.min), col = 'darkgreen', lty = 3, lwd = 2) 
  text(log(lra$lambda.min) - 1, r[2], col = 'darkgreen', 'chosen lambda')
  segments(log(lra$lambda.min), r[2] - 0.3 , log(lra$lambda.min) - 1, r[2], col = 'darkgreen')
}

In [ ]:
#::GMG::Let's what it looks like ...
options(repr.plot.width = 8, repr.plot.height = 6)
plot(cv_lasso$glmnet.fit, label=FALSE, xvar="lambda")
lbs_fun(cv_lasso)

### better coeff shrinkage plot with plotmo

In [ ]:
#::GMG::It's weird with those numeric labels (plot.glmnet)ç
#::NOTE::It seems there's an alternative with adhoc plotmo package
# https://stackoverflow.com/questions/30560689/adding-labels-on-curves-in-glmnet-plot-in-r
# https://stats.stackexchange.com/questions/133873/lasso-plot-label-lines-with-names-using-glmnet
# https://cran.r-project.org/web/packages/plotmo/plotmo.pdf#page=17
# plot_glmnet: Plot a glmnet model (Plot the coefficient paths of a glmnet model.)
# http://www.milbo.org/doc/plotres-notes.pdf#page=7
options(repr.plot.width = 8, repr.plot.height = 6)
plotmo::plot_glmnet(x = cv_lasso$glmnet.fit, xvar = 'lambda', label = TRUE)

### model

**::GMG::** In STHDA a new model is trained with the best lambdas found out with the CV approach. Is it really necessary to do that? Does not `cv.glmnet()` already have provided such a model?

In [ ]:
#::GMG::Cross-validated model with ... lambda.min or lambda.1se?
# https://www.rdocumentation.org/packages/glmnet/versions/2.0-18/topics/cv.glmnet
# https://cran.r-project.org/web/packages/glmnet/glmnet.pdf pp. 6, 
# glmnet.fit: a fitted glmnet object for the full data.
str(cv_lasso$glmnet.fit)

In [ ]:
#::GMG::Let's see the regression coefficients
coef(cv_lasso)

In [ ]:
#::GMG:: (STHDA) Fit the final model on the training data and lambda.min
model_lambda_min <- glmnet(
    X_train, 
    y_train, 
    alpha = 1,                     #::NOTE:: L1 LASSO regularization
    family = "binomial",
    lambda = cv_lasso$lambda.min   #::NOTE::Lambda min for L1
)

In [ ]:
# Display regression coefficients
coef(model_lambda_min)

In [ ]:
#::GMG::Final model with lambda.1se (this is the only way to get it this time)
model_lambda_1se <- glmnet(
    X_train, 
    y_train, 
    alpha = 1, 
    family = "binomial",
    lambda = cv_lasso$lambda.1se
)

In [ ]:
# Display regression coefficients
coef(model_lambda_1se)

**::GMG::** Looking at the `coef()` from `cv_lasso` and `model_lambda_1se` I conclude that the model returned as value (`cv_lasso$glmnet.fit` reported in the documentation) is the simpler regularized L1 model indeed! 

### predict

In [ ]:
#::GMG::Make predictions on the test data using the new trained model (STHDA)
probabilities_lambda_min <- predict(model_lambda_min, newx = X_test)
predicted_classes_lambda_min <- ifelse(probabilities_lambda_min > 0.5, "pos", "neg")
#::NOTE::For the P/R Curve function
predicted_classes_lambda_min_num <- ifelse(probabilities_lambda_min > 0.5, 1, 0)

In [ ]:
#::GMG::Make prediction on test data on the test data using the lambda 1se model
probabilities_1se <- predict(model_lambda_1se, newx = X_test)
predicted_classes_1se <- ifelse(probabilities_1se > 0.5, "pos", "neg")
predicted_classes_1se_num <- ifelse(probabilities_1se > 0.5, 1, 0)

In [ ]:
#::GMG::Predict using the cross-validated model
# https://cran.r-project.org/web/packages/glmnet/glmnet.pdf pp. 18
# https://www.rdocumentation.org/packages/glmnet/versions/2.0-18/topics/predict.cv.glmnet
# This function makes predictions from a cross-validated glmnet model, using the stored "glmnet.fit" object, 
# and the optimal value chosen for lambda.
#::NOTE::So as long as I don't want the simpler model with lambda.1se I can predict with this!
probabilities_cv_lasso <- predict(cv_lasso, newx = X_test)
predicted_classes_cv_lasso <- ifelse(probabilities_cv_lasso > 0.5, "pos", "neg")
#::NOTE::For the P/R Curve function
predicted_classes_cv_lasso_num <- ifelse(probabilities_cv_lasso > 0.5, 1, 0)

In [ ]:
probabilities_lambda_min

### evaluation: confusion matrix and metrics

In [ ]:
#::GMG::'Raw' Model accuracy
#::NOTE::observed_classes <- test_data$diabetes
cat('Accuracy L1 Lambda Min Model: ', 
    mean(predicted_classes_lambda_min == observed_classes),
   '\nAccuracy L1 Lambda 1se (Simpler) Model: ', 
    mean(predicted_classes_1se == observed_classes),
   '\nAccuracy CV Lasso Model: ', 
    mean(predicted_classes_cv_lasso == observed_classes)
)

In [ ]:
#::GMG::Confusion Matrix
cf_lambda_min <- confusion_matrix(true = test_data$diabetes, pred = predicted_classes_lambda_min)
#cf_lambda_min <- MLmetrics::ConfusionMatrix(y_true = test_data$diabetes, 
#                                            y_pred = predicted_classes_lambda_min)
#print(cf_lambda_min)
print(cf_lambda_min$cm)

In [ ]:
#::GMG::Confusion Matrix
cf_lambda_1se <- confusion_matrix(true = test_data$diabetes, pred = predicted_classes_1se)
print(cf_lambda_1se$cm)

In [ ]:
#::GMG::Confusion Matrix
cf_cv_lasso <- confusion_matrix(true = test_data$diabetes, pred = predicted_classes_cv_lasso)
print(cf_cv_lasso$cm)

In [ ]:
#::GMG::Metrics
# https://stackoverflow.com/questions/3443687/formatting-decimal-places-in-r
cat(sep='', "Accuracy: ", format(round(cf_full$accuracy, 2), nsmall = 2), 
    ", Balanced Accuracy ", format(round(cf_full$balanced_accuracy, 2), nsmall = 2))

In [ ]:
cat(sep = '', "Precision: ", format(round(cf_full$precision, 2), nsmall = 2), 
    ", Recall: ", format(round(cf_full$recall, 2), nsmall = 2),
    ", Fl: ", format(round(cf_full$f1, 2), nsmall = 2)
   )

In [ ]:
cat(sep='', "Specificity: ", format(round(cf_full$specificity, 2), nsmall = 2), 
    ", Sensitivity ", format(round(cf_full$recall, 2), nsmall = 2))

In [ ]:
#::GMG::Confusion Matrix of the 'simpler' model ... no difference!
cf_simpler <- confusion_matrix(true = test_data$diabetes, pred = predicted_class_simpler)
print(cf_simpler$cm)

### evaluation: P/R Curve

In [ ]:
#::GMG::Compute the curve
# https://cran.r-project.org/web/packages/PRROC/PRROC.pdf pp. 7
# Soft-labeled classification problems:
# Each data point belongs to both of the two classes with a certain probability, where for each datapoint, these 
# two probabilities add up to 1.  In this case, the classification scores for all data points need to be provided 
# only once as scores.class0 and only the positive/foreground weights for each data point need to be provided in
# weights.class0, while the converse probability for the negative class is automatically set to 
# weights.class1 = 1.0 - weights.class0.
pr_curve_full <- PRROC::pr.curve(
                scores.class0 = predicted_class_num, 
                weights.class0 = probabilities_full,
                min.compute = T, 
                max.compute = T,
                rand.compute = T,
                curve = T
)

In [ ]:
#::GMG::See all things within the pr_1 object
print(pr_curve_full)

In [ ]:
#::GMG::Plot the curve
# https://cran.r-project.org/web/packages/PRROC/PRROC.pdf pp 3-4
options(repr.plot.width = 8, repr.plot.height = 6)
plot(pr_curve_full, rand.plot = T)

**::PENDING::** <span style="color:red">Discussion here</span>

**::NOTE::** Is this right?

In [ ]:
ls()